# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
! wget ! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-24 18:46:52--  http://!/
Resolving ! (!)... failed: Name or service not known.
wget: unable to resolve host address ‘!’
--2024-03-24 18:46:52--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2024-03-24 18:46:52--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-24 18:46:52 (43.4 MB/s) - ‘iris.csv’ saved [3975/3975]

FINISHED --2024-03-24 18:46:52--
Total wall clock time: 0.2s
Downloaded: 1 files, 3.9K in 0s (43.4 MB/s)


Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [6]:
label_uji = data_uji.pop('variety')

In [7]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
84,5.4,3.0,4.5,1.5
11,4.8,3.4,1.6,0.2
149,5.9,3.0,5.1,1.8
132,6.4,2.8,5.6,2.2
124,6.7,3.3,5.7,2.1


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data.

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [8]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key]=prior[key]/n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [9]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Setosa': 0.3416666666666667, 'Versicolor': 0.3333333333333333, 'Virginica': 0.325})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [10]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_columns_name = input_data.columns[-1]
  list_class = set(input_data[class_columns_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].mean()
      std[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].std()
  return (rata2, std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [11]:
import math
def hitung_likelihood_gaussian(data, rata2, std):
  hasil = (1/math.sqrt(2*math.pi*(std**2))) * math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas.

In [12]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  (rata2, std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [13]:
model = training_naive_bayes_gaussian(data_latih)

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [14]:
def testing_naive_bayes_gaussian(model, data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class] * hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
    posterior[a_class]=posterior[a_class]*prior[a_class]
  kelas_uji = max(posterior, key=posterior.get)
  return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [15]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Versicolor
Versicolor


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [16]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model,data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total==label_uji))

Total prediksi benar:  29


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [17]:
from google.colab import files
kontak = files.upload()
for fn in kontak.keys():
  print('Nama file "{name}" dengan panjang {length} bytes'.format(name=fn, length=len(kontak[fn])))

Saving car_sample (1).csv to car_sample (1).csv
Nama file "car_sample (1).csv" dengan panjang 43263 bytes


In [18]:
import pandas as pd
data = pd.read_csv('car_sample (1).csv')
data.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [19]:
tugas_latih, tugas_uji = train_test_split(data, test_size = 0.3)
print(tugas_latih.shape[0])
print(tugas_uji.shape[0])

1209
519


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [20]:
tugas_label_uji = tugas_uji.pop ('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [21]:
prior_tugas = hitung_prior(tugas_latih['class'])
print(prior_tugas)

Counter({'unacc': 0.6906534325889164, 'acc': 0.22663358147229115, 'vgood': 0.043010752688172046, 'good': 0.03970223325062035})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [22]:
def hitung_likelihood_multinomial(data_latih):
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = np.unique(data_latih[class_column_name])
  likelihood = {}

  for a_class in list_class:
    for column in list_columns:
      grouped = data_latih.groupby(column).size()
      unique_val = grouped.index
      for col_ in unique_val :
        likelihood[(column, col_, a_class)] = 0

  for a_class in list_class:
    for column in list_columns:
      grouped = data_latih[data_latih[class_column_name] == a_class].groupby(column).size()
      unique_val = grouped.index
      for col_ in unique_val:
        numerator = grouped[col_]
        denominator = data_latih.groupby('class').size()[a_class]
        likelihood[(column, col_, a_class)] = numerator / denominator
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [23]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'high', 'acc'): 0.2664233576642336, ('buying', 'low', 'acc'): 0.25547445255474455, ('buying', 'med', 'acc'): 0.2737226277372263, ('buying', 'vhigh', 'acc'): 0.20437956204379562, ('maint', 'high', 'acc'): 0.2773722627737226, ('maint', 'low', 'acc'): 0.2518248175182482, ('maint', 'med', 'acc'): 0.28832116788321166, ('maint', 'vhigh', 'acc'): 0.18248175182481752, ('lug_boot', 'big', 'acc'): 0.35766423357664234, ('lug_boot', 'med', 'acc'): 0.354014598540146, ('lug_boot', 'small', 'acc'): 0.28832116788321166, ('safety', 'high', 'acc'): 0.5401459854014599, ('safety', 'low', 'acc'): 0, ('safety', 'med', 'acc'): 0.45985401459854014, ('buying', 'high', 'good'): 0, ('buying', 'low', 'good'): 0.6458333333333334, ('buying', 'med', 'good'): 0.3541666666666667, ('buying', 'vhigh', 'good'): 0, ('maint', 'high', 'good'): 0, ('maint', 'low', 'good'): 0.6666666666666666, ('maint', 'med', 'good'): 0.3333333333333333, ('maint', 'vhigh', 'good'): 0, ('lug_boot', 'big', 'good'): 0.29166666666666

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [24]:
def training_naive_bayes_multinomial(data_latih):
  class_column_name = data_latih.columns[-1]
  list_columns = tugas_latih.columns[:-1]
  model = {}
  model['prior'] = hitung_prior(data_latih[class_column_name])
  model['likelihood'] = hitung_likelihood_multinomial(data_latih)
  model['list_class'] = set(tugas_latih[class_column_name])
  model['list_columns'] = data_latih.columns[:-1]

  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [25]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)
print(model_tugas)

{'prior': Counter({'unacc': 0.6906534325889164, 'acc': 0.22663358147229115, 'vgood': 0.043010752688172046, 'good': 0.03970223325062035}), 'likelihood': {('buying', 'high', 'acc'): 0.2664233576642336, ('buying', 'low', 'acc'): 0.25547445255474455, ('buying', 'med', 'acc'): 0.2737226277372263, ('buying', 'vhigh', 'acc'): 0.20437956204379562, ('maint', 'high', 'acc'): 0.2773722627737226, ('maint', 'low', 'acc'): 0.2518248175182482, ('maint', 'med', 'acc'): 0.28832116788321166, ('maint', 'vhigh', 'acc'): 0.18248175182481752, ('lug_boot', 'big', 'acc'): 0.35766423357664234, ('lug_boot', 'med', 'acc'): 0.354014598540146, ('lug_boot', 'small', 'acc'): 0.28832116788321166, ('safety', 'high', 'acc'): 0.5401459854014599, ('safety', 'low', 'acc'): 0, ('safety', 'med', 'acc'): 0.45985401459854014, ('buying', 'high', 'good'): 0, ('buying', 'low', 'good'): 0.6458333333333334, ('buying', 'med', 'good'): 0.3541666666666667, ('buying', 'vhigh', 'good'): 0, ('maint', 'high', 'good'): 0, ('maint', 'low',

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [26]:
def testing_naive_bayes_multinomial(model,data_uji):
  likelihood = model['likelihood']
  list_class = model['list_class']
  list_columns = model['list_columns']
  prior = model['prior']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      col_ = str(data_uji[column])
      posterior[a_class] = posterior[a_class] * likelihood[column, col_, a_class]
    posterior[a_class] = posterior[a_class] * prior[a_class]
  kelas_uji = max(posterior, key = posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [27]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

acc
acc


Lakukan pengujian terhadap semua data uji.

In [28]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  374


In [29]:
len(tugas_label_uji)

519

In [31]:
akurasi = sum(prediksi_total == tugas_label_uji) / len(tugas_label_uji)

print("Akurasi: ", akurasi)
print("Total prediksi benar: ", sum(prediksi_total == tugas_label_uji))
print("Total data uji: ", len(tugas_label_uji))

Akurasi:  0.720616570327553
Total prediksi benar:  374
Total data uji:  519


Pada tugas ini, kita akan mengklasifikasi data mobil berdasarkan fitur seperti harga beli, biaya perawatan, ukuran bagasi, dan skor keamanan. Disini kita mengugnakan Naive Bayes Multinomial. Untuk langkah langkah nya adalah sebagai berikut,

1.   Membagi data menjadi data latih (70%) dan data uji (30%).
2.   Menghitung prior dan likelihood dari data latih.
3.   Membangun model Naive Bayes Multinomial.
4.   Menguji model pada data uji.
5.   Menghitung akurasi model.

Hasil :
*   Akurasi : 72.06%
*   Prediksi Benar : 374
*   Prediksi Salah  : 145


